In [17]:
# Data Type Analysis for POD and NAV
import pandas as pd
import numpy as np

print("=== POD DATA ANALYSIS ===")
print()

# Load POD data
try:
    pod = pd.read_csv('open purchase2.csv')
    print("POD Data Shape:", pod.shape)
    print("\nPOD Columns:")
    print(pod.columns.tolist())
    print("\nPOD Data Types:")
    print(pod.dtypes)
    print("\nPOD First 3 rows:")
    print(pod.head(3))
    print("\nPOD Sample QB Num values:")
    print(pod['QB Num'].head(10).tolist())
    print("\nPOD Sample Item values:")
    print(pod['Item'].head(10).tolist())
except FileNotFoundError:
    print("POD file not found - 'open purchase2.csv'")
except Exception as e:
    print(f"Error loading POD: {e}")


=== POD DATA ANALYSIS ===

POD Data Shape: (465, 8)

POD Columns:
['Order Date', 'QB Num', 'P. O. #', 'Name', 'Deliv Date', 'Qty(+)', 'Item', 'Inventory Site']

POD Data Types:
Order Date         object
QB Num             object
P. O. #            object
Name               object
Deliv Date         object
Qty(+)            float64
Item               object
Inventory Site     object
dtype: object

POD First 3 rows:
   Order Date      QB Num      P. O. #                        Name  \
0  2025/07/24  POD-251046  SO-20251050  Neousys Technology Incorp.   
1  2025/07/24  POD-251047  SO-20251051  Neousys Technology Incorp.   
2  2025/07/24  POD-251048  SO-20251052  Neousys Technology Incorp.   

   Deliv Date  Qty(+)                        Item Inventory Site  
0  2025/10/03    34.0  AccsyBx-6AntiVG-POC-551VTC      Drop Ship  
1  2025/11/21    24.0  AccsyBx-6AntiVG-POC-551VTC      Drop Ship  
2  2025/12/19    29.0  AccsyBx-6AntiVG-POC-551VTC      Drop Ship  

POD Sample QB Num values:
['POD-

In [18]:
print("=== NAV DATA ANALYSIS ===")
print()

# Load NAV data
try:
    nav = pd.read_csv('NAV1.csv')
    print("NAV Data Shape:", nav.shape)
    print("\nNAV Columns:")
    print(nav.columns.tolist())
    print("\nNAV Data Types:")
    print(nav.dtypes)
    print("\nNAV First 3 rows:")
    print(nav.head(3))
    print("\nNAV Sample QB Num values:")
    print(nav['QB Num'].head(10).tolist())
    print("\nNAV Sample Item values:")
    print(nav['Item'].head(10).tolist())
    print("\nNAV Sample Ship Date values:")
    print(nav['Ship Date'].head(10).tolist())
    print("\nNAV Sample No. values:")
    print(nav['No.'].head(10).tolist())
except FileNotFoundError:
    print("NAV file not found - 'NAV1.csv'")
except Exception as e:
    print(f"Error loading NAV: {e}")


=== NAV DATA ANALYSIS ===

NAV Data Shape: (230, 5)

NAV Columns:
['Remark', 'QB Num', 'Item', 'Qty(+)', 'Ship Date']

NAV Data Types:
Remark       object
QB Num       object
Item         object
Qty(+)       object
Ship Date    object
dtype: object

NAV First 3 rows:
       Remark            QB Num                    Item Qty(+)   Ship Date
0  LO25080011  For NTA_CoastIPC              RGS-8805GC      1   2025/9/10
1  SO25050035        POD-250648  SEMIL-1708-i7OCG9-FF04     10  2025/11/19
2  SO25050035        POD-250648   Cbl-M12S4F-OW4-180CM1     10  2025/11/19

NAV Sample QB Num values:
['For NTA_CoastIPC', 'POD-250648', 'POD-250648', 'POD-250649', 'POD-250649', 'POD-250650', 'POD-250650', 'POD-251059', 'POD-251059', 'POD-251060']

NAV Sample Item values:
['RGS-8805GC', 'SEMIL-1708-i7OCG9-FF04', 'Cbl-M12S4F-OW4-180CM1', 'SEMIL-1708-i7OCG9-FF04', 'Cbl-M12S4F-OW4-180CM1', 'SEMIL-1708-i7OCG9-FF04', 'Cbl-M12S4F-OW4-180CM1', 'POC-330-AEI01-100', 'Cbl-DB9F-3DB9M-15CM2', 'POC-330-AEI01-100']

In [20]:
print("=== KEY COLUMNS ANALYSIS ===")
print()

# Analyze key columns for matching
if 'pod' in locals() and 'nav' in locals():
    print("QB Num matching analysis:")
    print(f"POD unique QB Nums: {pod['QB Num'].nunique()}")
    print(f"NAV unique QB Nums: {nav['QB Num'].nunique()}")
    
    # Check for common QB Nums
    pod_qb = set(pod['QB Num'].dropna())
    nav_qb = set(nav['QB Num'].dropna())
    common_qb = pod_qb.intersection(nav_qb)
    print(f"Common QB Nums: {len(common_qb)}")
    
    print("\nItem matching analysis:")
    print(f"POD unique Items: {pod['Item'].nunique()}")
    print(f"NAV unique Items: {nav['Item'].nunique()}")
    
    # Check for common Items
    pod_items = set(pod['Item'].dropna())
    nav_items = set(nav['Item'].dropna())
    common_items = pod_items.intersection(nav_items)
    print(f"Common Items: {len(common_items)}")
    
    print("\nPre-installed items analysis (S-prefix):")
    s_items = nav[nav['Remark'].astype(str).str.startswith('S', na=False)]
    print(f"NAV items with S-prefix: {s_items['Item'].nunique()}")
    print(f"Sample S-prefix items: {s_items['Item'].head(5).tolist()}")
    
    print("\nShip Date analysis:")
    print(f"NAV unique Ship Dates: {nav['Ship Date'].nunique()}")
    print(f"Sample Ship Dates: {sorted(nav['Ship Date'].dropna().unique())[:5]}")
else:
    print("Cannot analyze - data not loaded properly")


=== KEY COLUMNS ANALYSIS ===

QB Num matching analysis:
POD unique QB Nums: 159
NAV unique QB Nums: 98
Common QB Nums: 91

Item matching analysis:
POD unique Items: 244
NAV unique Items: 146
Common Items: 116

Pre-installed items analysis (S-prefix):
NAV items with S-prefix: 140
Sample S-prefix items: ['SEMIL-1708-i7OCG9-FF04', 'Cbl-M12S4F-OW4-180CM1', 'SEMIL-1708-i7OCG9-FF04', 'Cbl-M12S4F-OW4-180CM1', 'SEMIL-1708-i7OCG9-FF04']

Ship Date analysis:
NAV unique Ship Dates: 16
Sample Ship Dates: [')', '2025/10/16', '2025/10/22', '2025/10/29', '2025/10/8']


In [ ]:
# Enhanced POD Processing
print("=== ENHANCED POD PROCESSING ===")
print()

# Ensure we have the data loaded
if 'pod' not in locals() or 'nav' not in locals():
    print("Loading data first...")
    pod = pd.read_csv('open purchase2.csv')
    nav = pd.read_csv('NAV1.csv')

# Debug: Check NAV columns
print("NAV columns:", nav.columns.tolist())
print("NAV sample data:")
print(nav.head(2))

# Create a copy of POD for processing
enhanced_pod = pod.copy()

# Add new columns
enhanced_pod['Barebone/Pre-installed'] = ''
enhanced_pod['Ship Date'] = ''
enhanced_pod['Status'] = ''

print(f"Starting with {len(enhanced_pod)} POD records")
print(f"NAV has {len(nav)} records")

# Check if 'No.' column exists, if not, look for alternative column
if 'No.' in nav.columns:
    s_prefix_items = set(nav[nav['No.'].astype(str).str.startswith('S', na=False)]['Item'].unique())
    print(f"Found {len(s_prefix_items)} pre-installed items in NAV using 'No.' column")
elif 'Remark' in nav.columns:
    # Check if Remark column contains S-prefix information
    s_prefix_items = set(nav[nav['Remark'].astype(str).str.startswith('S', na=False)]['Item'].unique())
    print(f"Found {len(s_prefix_items)} pre-installed items in NAV using 'Remark' column")
else:
    print("No suitable column found for S-prefix detection. All items will be treated as Barebone.")
    s_prefix_items = set()

# Lists to track partial shipments and missing data
partial_shipments = []
missing_nav_data = []

print("\nProcessing POD items...")


=== ENHANCED POD PROCESSING ===

NAV columns: ['Remark', 'QB Num', 'Item', 'Qty(+)', 'Ship Date']
NAV sample data:
       Remark            QB Num                    Item Qty(+)   Ship Date
0  LO25080011  For NTA_CoastIPC              RGS-8805GC      1   2025/9/10
1  SO25050035        POD-250648  SEMIL-1708-i7OCG9-FF04     10  2025/11/19
Starting with 465 POD records
NAV has 230 records
Found 140 pre-installed items in NAV using 'Remark' column

Processing POD items...


In [ ]:
# Process each POD item
enhanced_pod_list = []

for idx, pod_row in enhanced_pod.iterrows():
    item = pod_row['Item']
    qb_num = pod_row['QB Num']
    
    # Check if item is pre-installed (only if we found S-prefix items)
    if len(s_prefix_items) > 0 and item in s_prefix_items:
        item_type = 'Pre-installed'
        # Look up by QB Num only for pre-installed items
        nav_matches = nav[(nav['QB Num'] == qb_num)]
    else:
        item_type = 'Barebone'
        # Look up by both Item and QB Num for barebone items
        nav_matches = nav[(nav['Item'] == item) & (nav['QB Num'] == qb_num)]
    
    # Handle based on number of NAV matches
    if len(nav_matches) == 0:
        # No NAV data found
        new_row = pod_row.copy()
        new_row['Barebone/Pre-installed'] = item_type
        new_row['Ship Date'] = ''
        new_row['Status'] = 'No NAV Data'
        enhanced_pod_list.append(new_row)
        missing_nav_data.append(item)
        
    elif len(nav_matches) == 1:
        # Single shipment
        nav_row = nav_matches.iloc[0]
        new_row = pod_row.copy()
        new_row['Barebone/Pre-installed'] = item_type
        new_row['Ship Date'] = nav_row['Ship Date']
        new_row['Status'] = 'Complete'
        enhanced_pod_list.append(new_row)
        
    else:
        # Multiple shipments (partial shipment)
        unique_ship_dates = nav_matches['Ship Date'].unique()
        original_qty = pod_row['Qty(+)'] if 'Qty(+)' in pod_row else 1
        
        for i, ship_date in enumerate(unique_ship_dates, 1):
            new_row = pod_row.copy()
            new_row['QB Num'] = f"{qb_num}(P{i})"
            new_row['Barebone/Pre-installed'] = item_type
            new_row['Ship Date'] = ship_date
            new_row['Status'] = 'Partial'
            
            # Distribute quantity equally across partial shipments
            if isinstance(original_qty, (int, float)):
                new_row['Qty(+)'] = original_qty / len(unique_ship_dates)
            else:
                new_row['Qty(+)'] = original_qty
                
            enhanced_pod_list.append(new_row)
        
        partial_shipments.append({
            'original_qb': qb_num,
            'item': item,
            'type': item_type,
            'ship_dates': list(unique_ship_dates),
            'count': len(unique_ship_dates)
        })

print(f"Processed {len(enhanced_pod_list)} records")
print(f"Found {len(partial_shipments)} items with partial shipments")
print(f"Found {len(missing_nav_data)} items with missing NAV data")


Processed 499 records
Found 176 items with partial shipments
Found 257 items with missing NAV data


In [ ]:
# Create final enhanced POD DataFrame
enhanced_pod_final = pd.DataFrame(enhanced_pod_list)

# Reorder columns for better readability
column_order = ['QB Num', 'Item', 'Barebone/Pre-installed', 'Ship Date', 'Qty(+)', 'Order Date', 'Status']
if 'Inventory Site' in enhanced_pod_final.columns:
    column_order.insert(-2, 'Inventory Site')
if 'P. O. #' in enhanced_pod_final.columns:
    column_order.insert(-2, 'P. O. #')
if 'Name' in enhanced_pod_final.columns:
    column_order.insert(-2, 'Name')

# Select only existing columns
existing_columns = [col for col in column_order if col in enhanced_pod_final.columns]
enhanced_pod_final = enhanced_pod_final[existing_columns]

print("=== ENHANCED POD RESULTS ===")
print(f"Final dataset shape: {enhanced_pod_final.shape}")
print(f"\nColumn order: {existing_columns}")
print(f"\nFirst 5 rows:")
print(enhanced_pod_final.head())

print(f"\nStatus distribution:")
print(enhanced_pod_final['Status'].value_counts())

print(f"\nType distribution:")
print(enhanced_pod_final['Barebone/Pre-installed'].value_counts())


=== ENHANCED POD RESULTS ===
Final dataset shape: (499, 10)

Column order: ['QB Num', 'Item', 'Barebone/Pre-installed', 'Ship Date', 'Qty(+)', 'Inventory Site', 'P. O. #', 'Name', 'Order Date', 'Status']

First 5 rows:
           QB Num                             Item Barebone/Pre-installed  \
0      POD-251046       AccsyBx-6AntiVG-POC-551VTC               Barebone   
1      POD-251047       AccsyBx-6AntiVG-POC-551VTC               Barebone   
2      POD-251048       AccsyBx-6AntiVG-POC-551VTC               Barebone   
3      POD-251338  AccsyBx-Cardholder-9160GC-2000E          Pre-installed   
4  POD-251279(P1)              AccsyBx-FAN-NRU-100          Pre-installed   

   Ship Date  Qty(+) Inventory Site      P. O. #                        Name  \
0               34.0      Drop Ship  SO-20251050  Neousys Technology Incorp.   
1               24.0      Drop Ship  SO-20251051  Neousys Technology Incorp.   
2               29.0      Drop Ship  SO-20251052  Neousys Technology Incorp.  

In [ ]:
# Generate Reports
print("=== PROCESSING REPORTS ===")
print()

# Partial Shipments Report
if partial_shipments:
    print("🚢 PARTIAL SHIPMENTS REPORT:")
    print("-" * 50)
    for shipment in partial_shipments:
        print(f"• {shipment['original_qb']} ({shipment['item']}) - {shipment['type']}")
        print(f"  Split into {shipment['count']} partial shipments:")
        for i, date in enumerate(shipment['ship_dates'], 1):
            print(f"    - {shipment['original_qb']}(P{i}) on {date}")
        print()
else:
    print("✅ No partial shipments found")

print()

# Missing NAV Data Report
if missing_nav_data:
    # Filter out NaN values and convert to strings
    unique_missing = [str(item) for item in set(missing_nav_data) if pd.notna(item)]
    print("⚠️  MISSING NAV DATA REPORT:")
    print("-" * 50)
    print(f"Items without corresponding NAV records ({len(unique_missing)} unique items):")
    for item in sorted(unique_missing):
        print(f"  • {item}")
    print()
else:
    print("✅ All items have corresponding NAV data")

print()

# Summary Statistics
print("📊 SUMMARY STATISTICS:")
print("-" * 50)
print(f"Original POD records: {len(pod)}")
print(f"Enhanced POD records: {len(enhanced_pod_final)}")
print(f"Records added due to partial shipments: {len(enhanced_pod_final) - len(pod)}")
print(f"Pre-installed items: {len(enhanced_pod_final[enhanced_pod_final['Barebone/Pre-installed'] == 'Pre-installed'])}")
print(f"Barebone items: {len(enhanced_pod_final[enhanced_pod_final['Barebone/Pre-installed'] == 'Barebone'])}")


=== PROCESSING REPORTS ===

🚢 PARTIAL SHIPMENTS REPORT:
--------------------------------------------------
• POD-251279 (AccsyBx-FAN-NRU-100) - Pre-installed
  Split into 1 partial shipments:
    - POD-251279(P1) on 2025/10/8

• POD-251333 (Ant-RP_SMAM-WiFi-108MM) - Pre-installed
  Split into 1 partial shipments:
    - POD-251333(P1) on 2025/10/22

• POD-251383 (Ant-RP_SMAM-WiFi-108MM) - Pre-installed
  Split into 1 partial shipments:
    - POD-251383(P1) on nan

• POD-251164 (Ant-RP_SMAM-WiFi-196MM1) - Pre-installed
  Split into 1 partial shipments:
    - POD-251164(P1) on 2025/10/16

• POD-251180 (Ant-RP_SMAM-WiFi-196MM1) - Pre-installed
  Split into 1 partial shipments:
    - POD-251180(P1) on 2025/10/8

• POD-251181 (Ant-RP_SMAM-WiFi-196MM1) - Pre-installed
  Split into 1 partial shipments:
    - POD-251181(P1) on 2025/10/16

• POD-251182 (Ant-RP_SMAM-WiFi-196MM1) - Pre-installed
  Split into 2 partial shipments:
    - POD-251182(P1) on 2025/10/16
    - POD-251182(P2) on 2025/10/29